In [2]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from skimage import io, segmentation, color
from openslide import OpenSlide

In [3]:
data_path = Path("/mnt/nas6/data/CPTAC")
image_path = list(data_path.rglob("*.svs"))

In [4]:
wsi_image = OpenSlide(image_path[0])
test_image = wsi_image.get_thumbnail((wsi_image.dimensions[0] // 100, wsi_image.dimensions[1] // 100))

In [5]:
# Apply SLIC superpixel segmentation
num_segments = 200  # Adjust for granularity
segments = segmentation.slic(test_image, n_segments=num_segments, compactness=10, sigma=1, start_label=1)

# Overlay superpixels on the original image
superpixel_boundaries = segmentation.mark_boundaries(test_image, segments)

# Display the result
plt.figure(figsize=(8, 8))
plt.imshow(superpixel_boundaries)
plt.axis('off')
plt.show()

FileNotFoundError: No such file: '/home/valentin/workspaces/histolung/notebooks/your_image.jpg'